## Test del tool su 

In [0]:
! git clone https://github.com/Meyke/ProgettoSIIML.git
! git clone --recursive https://github.com/metalbubble/TRN-pytorch

Cloning into 'ProgettoSIIML'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 903 (delta 9), reused 15 (delta 8), pack-reused 883
Receiving objects: 100% (903/903), 141.53 MiB | 31.91 MiB/s, done.
Resolving deltas: 100% (224/224), done.
Cloning into 'TRN-pytorch'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 263 (delta 0), reused 6 (delta 0), pack-reused 257
Receiving objects: 100% (263/263), 69.11 KiB | 3.64 MiB/s, done.
Resolving deltas: 100% (145/145), done.
Submodule 'model_zoo' (https://github.com/yjxiong/tensorflow-model-zoo.torch) registered for path 'model_zoo'
Cloning into '/content/TRN-pytorch/model_zoo'...
remote: Enumerating objects: 621, done.        
remote: Total 621 (delta 0), reused 0 (delta 0), pack-reused 621        
Receiving objects: 100% (621/621), 43.66 MiB | 28.55

In [0]:
% cd /content/ProgettoSIIML/deeplearning/sports_recognition

/content/ProgettoSIIML/deeplearning/sports_recognition


In [0]:
#@title
import csv
with open('sports1.csv') as file1:
    reader1 = csv.reader(file1)
    sports1 = {rows[0]:rows[1] for rows in reader1}
with open('sports2.csv') as file2:
    reader2 = csv.reader(file2)
    sports2 = {rows[0]:rows[1] for rows in reader2}
with open('videos.csv') as file3:
    reader3 = csv.reader(file3)
    videos = {rows[0]:rows[1].title() for rows in reader3}
print(videos)

{'https://www.youtube.com/watch?v=1XHhmGVJn4U': 'Cycling', 'https://www.youtube.com/watch?v=W4LG1AglioQ': 'Cycling', 'https://www.youtube.com/watch?v=KoI81CdYEa8': 'Cycling', 'https://www.youtube.com/watch?v=6FWeSVv8UVs': 'Cycling', 'https://www.youtube.com/watch?v=H52FAOBfbV8': 'Cycling', 'https://www.youtube.com/watch?v=UbVoAIaLCak': 'Cycling', 'https://www.youtube.com/watch?v=Q13D3qe7veY': 'Cycling', 'https://www.youtube.com/watch?v=XFfOUGTUuzc': 'Cycling', 'https://www.youtube.com/watch?v=-NRvACRKDCQ': 'Cycling', 'https://www.youtube.com/watch?v=uDtdXmS2KiE': 'Cycling', 'https://www.youtube.com/watch?v=l0SFfiAPTE0': 'Cycling', 'https://www.youtube.com/watch?v=WqpVpSnmHMU': 'Cycling', 'https://www.youtube.com/watch?v=qI3Pm4YIn8o': 'Cycling', 'https://www.youtube.com/watch?v=14mASFd0J44': 'Cycling', 'https://www.youtube.com/watch?v=nSW_mm3dgv8': 'Cycling', 'https://www.youtube.com/watch?v=OCG43Jf-Gq0': 'Cycling', 'https://www.youtube.com/watch?v=EewTm3UoPHw': 'Cycling', 'https://www.

In [0]:
% cd /content/TRN-pytorch/

/content/TRN-pytorch


In [0]:
#@title
% cd pretrain
! ./download_models.sh

/content/TRN-pytorch/pretrain
--2019-04-14 17:11:00--  http://relation.csail.mit.edu/models/TRN_something_RGB_BNInception_TRNmultiscale_segment8_best.pth.tar
Resolving relation.csail.mit.edu (relation.csail.mit.edu)... 128.30.100.255
Connecting to relation.csail.mit.edu (relation.csail.mit.edu)|128.30.100.255|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52711275 (50M) [application/x-tar]
Saving to: ‘TRN_something_RGB_BNInception_TRNmultiscale_segment8_best.pth.tar’

TRN_something_RGB_B 100%[===================>]  50.27M  72.2MB/s    in 0.7s    

2019-04-14 17:11:01 (72.2 MB/s) - ‘TRN_something_RGB_BNInception_TRNmultiscale_segment8_best.pth.tar’ saved [52711275/52711275]

--2019-04-14 17:11:01--  http://relation.csail.mit.edu/models/TRN_something_RGB_BNInception_TRN_segment3_best.pth.tar
Resolving relation.csail.mit.edu (relation.csail.mit.edu)... 128.30.100.255
Connecting to relation.csail.mit.edu (relation.csail.mit.edu)|128.30.100.255|:80... connected.
HT

In [0]:
% cd /content/TRN-pytorch/model_zoo/bninception

/content/TRN-pytorch/model_zoo/bninception


In [0]:
%%writefile pytorch_load.py

import torch
from torch import nn
from .layer_factory import get_basic_layer, parse_expr
import torch.utils.model_zoo as model_zoo
import yaml


class BNInception(nn.Module):
    def __init__(self, model_path='model_zoo/bninception/bn_inception.yaml', num_classes=101,
                       weight_url='https://yjxiong.blob.core.windows.net/models/bn_inception-9f5701afb96c8044.pth'):
        super(BNInception, self).__init__()

        manifest = yaml.load(open(model_path))

        layers = manifest['layers']

        self._channel_dict = dict()

        self._op_list = list()
        for l in layers:
            out_var, op, in_var = parse_expr(l['expr'])
            if op != 'Concat':
                id, out_name, module, out_channel, in_name = get_basic_layer(l,
                                                                3 if len(self._channel_dict) == 0 else self._channel_dict[in_var[0]],
                                                                             conv_bias=True)

                self._channel_dict[out_name] = out_channel
                setattr(self, id, module)
                self._op_list.append((id, op, out_name, in_name))
            else:
                self._op_list.append((id, op, out_var[0], in_var))
                channel = sum([self._channel_dict[x] for x in in_var])
                self._channel_dict[out_var[0]] = channel

        #self.load_state_dict(torch.utils.model_zoo.load_url(weight_url))

    def forward(self, input):
        data_dict = dict()
        data_dict[self._op_list[0][-1]] = input

        def get_hook(name):

            def hook(m, grad_in, grad_out):
                print(name, grad_out[0].data.abs().mean())

            return hook
        for op in self._op_list:
            if op[1] != 'Concat' and op[1] != 'InnerProduct':
                data_dict[op[2]] = getattr(self, op[0])(data_dict[op[-1]])
                # getattr(self, op[0]).register_backward_hook(get_hook(op[0]))
            elif op[1] == 'InnerProduct':
                x = data_dict[op[-1]]
                data_dict[op[2]] = getattr(self, op[0])(x.view(x.size(0), -1))
            else:
                try:
                    data_dict[op[2]] = torch.cat(tuple(data_dict[x] for x in op[-1]), 1)
                except:
                    for x in op[-1]:
                        print(x,data_dict[x].size())
                    raise
        return data_dict[self._op_list[-1][2]]


class InceptionV3(BNInception):
    def __init__(self, model_path='model_zoo/bninception/inceptionv3.yaml', num_classes=101,
                 weight_url='https://yjxiong.blob.core.windows.net/models/inceptionv3-cuhk-0e09b300b493bc74c.pth'):
        super(InceptionV3, self).__init__(model_path=model_path, weight_url=weight_url, num_classes=num_classes)

Overwriting pytorch_load.py


In [0]:
cd /content/TRN-pytorch

/content/TRN-pytorch


In [0]:
!pip install pafy
!pip install youtube_dl
import cv2
import os
import csv
import numpy as np
import pafy
from PIL import Image
from collections import Counter

import json
import re
import argparse
import functools
import subprocess
import moviepy.editor as mpy
import ast
import torchvision
import torch.nn.parallel
import torch.optim
from models import TSN
import transforms
from torch.nn import functional as F

    100% |████████████████████████████████| 1.8MB 4.6MB/s 
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1949696/45929032 bytes (4.2%)5652480/45929032 bytes (12.3%)9175040/45929032 bytes (20.0%)12935168/45929032 bytes (28.2%)16637952/45929032 bytes (36.2%)20389888/45929032 bytes (44.4%)24059904/45929032 bytes (52.4%)27820032/45929032 bytes (60.6%)31440896/45929032 bytes (68.5%)35192832/45929032 bytes (76.6%)38969344/45929032 bytes (84.8%)42

In [0]:
def extract_frames(video_file, num_frames=8):
      try:
          os.makedirs(os.path.join(os.getcwd(), 'frames'))
      except OSError:
          pass

      output = subprocess.Popen(['ffmpeg', '-i', video_file],
                                stderr=subprocess.PIPE).communicate()
      # Search and parse 'Duration: 00:05:24.13,' from ffmpeg stderr.
      re_duration = re.compile('Duration: (.*?)\.')
      duration = re_duration.search(str(output[1])).groups()[0]
      temp = [int(i) for i in duration.split(":")]

      seconds = functools.reduce(lambda x, y: x * 60 + y,
                                 temp)
      rate = num_frames / float(seconds)

      output = subprocess.Popen(['ffmpeg', '-i', video_file,
                                 '-vf', 'fps={}'.format(rate),
                                 '-vframes', str(num_frames),
                                 '-loglevel', 'panic',
                                 'frames/%d.jpg']).communicate()
      frame_paths = sorted([os.path.join('frames', frame)
                            for frame in os.listdir('frames')])

      frames = load_frames(frame_paths)
      subprocess.call(['rm', '-rf', 'frames'])
      return frames


def load_frames(frame_paths, num_frames=8):
    frames = [Image.open(frame).convert('RGB') for frame in frame_paths]
    if len(frames) >= num_frames:
        return frames[::int(np.ceil(len(frames) / float(num_frames)))]
    else:
        raise ValueError('Video must have at least {} frames'.format(num_frames))


def render_frames(frames, prediction):
    rendered_frames = []
    for frame in frames:
        img = np.array(frame)
        height, width, _ = img.shape
        cv2.putText(img, prediction,
                    (1, int(height / 8)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255), 2)
        rendered_frames.append(img)
    return rendered_frames

def get_result(results):
    for result in results:
      if sports1.get(result) != None:
        return sports1.get(result)
    for result in results:
      if sports2.get(result) != None:
        return sports2.get(result)

# Get dataset categories.
categories_file = 'pretrain/moments_categories.txt'
categories = [line.rstrip() for line in open(categories_file, 'r').readlines()]
num_class = len(categories)

arch = 'InceptionV3'

# Load model.
net = TSN(num_class,
          8,
          'RGB',
          base_model= arch,
          consensus_type='TRNmultiscale',
          img_feature_dim=256, print_spec=False)

checkpoint = torch.load('pretrain/TRN_moments_RGB_InceptionV3_TRNmultiscale_segment8_best.pth.tar')
base_dict = {'.'.join(k.split('.')[1:]): v for k, v in list(checkpoint['state_dict'].items())}
net.load_state_dict(base_dict)
net.cuda().eval()

# Initialize frame transforms.
transform = torchvision.transforms.Compose([
    transforms.GroupOverSample(net.input_size, net.scale_size),
    transforms.Stack(roll=(arch in ['BNInception', 'InceptionV3'])),
    transforms.ToTorchFormatTensor(div=(arch not in ['BNInception', 'InceptionV3'])),
    transforms.GroupNormalize(net.input_mean, net.input_std),
])

Multi-Scale Temporal Relation Network Module in use ['8-frame relation', '7-frame relation', '6-frame relation', '5-frame relation', '4-frame relation', '3-frame relation', '2-frame relation']
Freezing BatchNorm2D except the first one.


In [0]:
corretti = 0
errati = 0
non_riconosciuti = 0
eccezzioni = 0
for key, value in videos.items():
  try:
      url = key
      vPafy = pafy.new(url)
      play = vPafy.getbest()
      video = play.download(filepath="video.mp4")


      # Obtain video frames
      frames = extract_frames("video.mp4", 8)


      # Make video prediction.
      data = transform(frames)
      input = data.view(-1, 3, data.size(1), data.size(2)).unsqueeze(0).cuda()

      with torch.no_grad():
          logits = net(input)
          h_x = torch.mean(F.softmax(logits, 1), dim=0).data
          probs, idx = h_x.sort(0, True)

      # Output the prediction.
      video_name = "video"
      predictions = []
      for i in range(0, 3):
          predictions.append(categories[idx[i]])


      


      if get_result(predictions) != None:
        if value == ast.literal_eval(get_result(predictions))[0]:
          corretti += 1
        else:
          errati += 1
      else:
        non_riconosciuti += 1
      
#      print("Corretti sono: " +str(corretti) + " Errati sono: " + str(errati) 
#            + " Non riconosciuti sono: " + str(non_riconosciuti) + " Eccezzioni sono: " + str(eccezzioni))
  except:
      eccezzioni += 1
      continue

In [0]:
print("Corretti sono: " +str(corretti) + ", Errati sono: " + str(errati) 
      + ", Non riconosciuti sono: " + str(non_riconosciuti) + ", Eccezzioni sono: " + str(eccezzioni))

Corretti sono: 19 Errati sono: 0 Non riconosciuti sono: 0 Eccezzioni sono: 1
